In [19]:

import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
import json
import autogen

from dotenv import load_dotenv

load_dotenv()

config_list = [
    {
        'model': 'gpt-4o',
        'api_key':  os.getenv("OPENAI_API_KEY"),
        'tags': ['tool', 'gpt-4'],
    },
    {
        'model': 'gemini-1.5-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.5-flash',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.0-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['gemini'],
    },
    {
        "model": "llama3.2:latest",  # Your local model name (e.g. "llama2", "mistral", "phi3")
        "base_url": "http://localhost:11434/v1/",
        "api_key": "ollama",  # Can be any non-empty string
        "tags": ["local", "ollama"],
    }
]

config_list_file_name = ".config_list"
with open(config_list_file_name, "w") as file:
    json.dump(config_list, file, indent=4)

gpt_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gpt-4"]})
gpt_llm_config = {"config_list": gpt_config_list, "timeout": 120}

gemini_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

ollama_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["local"]})
ollama_llm_config = {"config_list": gemini_config_list, "timeout": 500} # larger timeout

user_proxy = autogen.UserProxyAgent(
    name="User_Agent",
    system_message="A human user who needs help from other agents to collaboratively generate a code.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "working_dir",
        "use_docker": True  # Set to True if you want to use Docker for isolation
    },
    human_input_mode="NEVER"
)


inst1 = r"""1. Understand the scope of the user's query, and tools regarding debugging and testing the code's correctness and completeness.
2. Generate a professional code that serves the user's purpose.
3. Generate appropriate test cases for debugging the correctness and completeness.
4. Install any required dependencies (i.e. Python modules, tools) to resolve any bug (i.e. missing dependency) in the code.
    - Consider current system info while installing dependency, i.e. Mac, Linux, Windows etc.
    - When installing dependencies or performing system-level tasks, prefer using shell commands within a bash script.
5. Test the generated code and debug it until it produces expected results.
6. IMPORTANT: Before finishing, you MUST:
   a. Check if './code' directory exists, and create it if it doesn't using: os.makedirs('./code', exist_ok=True)
   b. Create a CLEAN version of the final working code that includes ONLY:
      - Required imports
      - The core algorithm/function implementations
      - Brief documentation comments (docstrings)
      - NO TEST CASES, assert statements, or debugging code
   c. Save ONLY this clean version to the './code' folder with a descriptive filename (e.g., 'algorithm_name.py')
   d. Verify the file was saved by listing the contents of the './code' directory
7. Delete any intermediate test and debug files. All test code should be removed from the final version.
8. Explicitly confirm that you've saved a clean, production-ready file to the './code' directory."""

inst2 = r"""Generate the code and debug until the code is correct and complete and finally
move the correct code into from your working directory ./code folder, with proper naming"""

coder = autogen.AssistantAgent(
    name="Expert_Coder_Agent",
    system_message=f"""You are an professional expert Python coder also a researcher and a planner. 
    When User_Agent asks to generate code,
    Your tasks is to:
       {inst1}
    """,
    llm_config={"config_list": gemini_config_list, "timeout": 120},
    # code_execution_config={
    #     "last_n_messages": 4,
    #     "executor": LocalCommandLineCodeExecutor(work_dir="working_dir")
    # },
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "working_dir",
        "use_docker": True  # Set to True if you want to use Docker for isolation
    }
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, coder],
    messages=[],
    max_round=20,
    speaker_selection_method='round_robin',
    allow_repeat_speaker=True
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config={"config_list": gemini_config_list},
    # llm_config={"config_list": ollama_config_list}
)

topic = """A SFTraining code for simple DNN dataset"""

user_proxy.initiate_chat(
    manager,
    message=f"""
    1. You need to generate a correct professional code for query: "{topic}". 
    2. Collaborate with Expert_Coder_Agent to generate, debug and improve the final code.
    3. IMPORTANT: Make sure the final code is saved in the './code' directory with a descriptive filename.
    4. Verify the code was properly saved by listing the contents of the './code' directory.
    5. If there is no more task pending at your side, return "TERMINATE" otherwise return "CONTINUE".
    """,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
)

User_Agent (to chat_manager):


    1. You need to generate a correct professional code for query: "A SFTraining code for simple DNN dataset". 
    2. Collaborate with Expert_Coder_Agent to generate, debug and improve the final code.
    3. IMPORTANT: Make sure the final code is saved in the './code' directory with a descriptive filename.
    4. Verify the code was properly saved by listing the contents of the './code' directory.
    5. If there is no more task pending at your side, return "TERMINATE" otherwise return "CONTINUE".
    

--------------------------------------------------------------------------------

Next speaker: Expert_Coder_Agent

Expert_Coder_Agent (to chat_manager):

```python
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification  # For demo dataset
from torch.utils.data import DataLoader, TensorDataset


# Create './code' directory if it doesn't

KeyboardInterrupt: 